In [1]:
# !pip install llama-index
# !pip install llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface

In [2]:
# from llama_index.llms.ollama import Ollama

# llm = Ollama(model="llama2", request_timeout=600) #base_url = 'http://localhost:11434',

# response = llm.complete("Do you know about Claude-3?") ## stream_complete() for streaming response

# response
# print(response.text)

In [3]:
# Importing required modules
from pypdf import PdfReader
from llama_index.core import VectorStoreIndex, Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama

# Creating a pdf reader object
reader = PdfReader('faq.pdf')

# Extracting text from all pages
all_text = ""
for page in reader.pages:
    all_text += page.extract_text() + "\n"

# Setting up the embedding model and LLM
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)
Settings.llm = Ollama(model="llama2", request_timeout=600)

# Creating a Document object
doc = Document(text=all_text)

# Creating the VectorStoreIndex
index = VectorStoreIndex.from_documents([doc])

/home/abhi/Documents/ChatBot/Llama2/v_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
retriever = index.as_retriever(similarity_top_k=2)

# Function to get response with LLM validation
def get_validated_response(question):
    # Retrieve relevant documents
    retrieved_nodes = retriever.retrieve(question)
    
    # Prepare context for LLM
    context = "\n".join([node.text for node in retrieved_nodes])
    
    # Prepare prompt for LLM
    prompt = f"""Given the following context and question, provide a accurate and concise answer. If the context doesn't contain enough information to answer the question confidently, state that clearly.

Context:
{context}

Question: {question}

Answer:"""

    # Use LLM to generate a validated response
    llm = Ollama(model="llama2", request_timeout=600, temperature=0.1)
    response = llm.complete(prompt)
    
    return response

# Example usage
# question = "i am a farmar, how can i apply for this scheme??"
# response = get_validated_response(question)
# print(response)

Pradhan Mantri Awas Yojana-Urban 2.0 (PMAY-U 2.0) is a central government scheme aimed at providing affordable housing to eligible urban families in India. The scheme was launched in 2024 and will be implemented for a period of 5 years until 2029. PMAY-U 2.0 provides financial assistance to states/UTs to construct, purchase or rent affordable houses in urban areas. The scheme focuses on providing housing opportunities to the Economically Weaker Section (EWS), Low Income Group (LIG), and Middle Income Group (MIG) segments.


In [ ]:
question = "i am a farmar, how can i apply for this scheme??"
response = get_validated_response(question)
print(response)